In [1]:
import pandas as pd
import numpy as np


MIMIC_DIR = "C:\\Users\\amits\\work\\datasets\\MIMIC-III-v1.4"

patients = pd.read_csv(f"{MIMIC_DIR}\\PATIENTS.csv.gz")
admissions = pd.read_csv(f"{MIMIC_DIR}\\ADMISSIONS.csv.gz",  parse_dates=['ADMITTIME'])
diagnoses_icd = pd.read_csv(f"{MIMIC_DIR}\\DIAGNOSES_ICD.csv.gz")
icustays = pd.read_csv(f"{MIMIC_DIR}\\ICUSTAYS.csv.gz")

micro = pd.read_csv(f"{MIMIC_DIR}\\MICROBIOLOGYEVENTS.csv.gz", parse_dates=['CHARTDATE'], low_memory=False) 
prescriptions = pd.read_csv(f"{MIMIC_DIR}\\PRESCRIPTIONS.csv.gz", parse_dates=['STARTDATE'], low_memory=False)


chartevents = pd.read_csv(f"{MIMIC_DIR}\\CHARTEVENTS.csv.gz", 
                          usecols=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUENUM'],
                          chunksize=1000000, # chartevents is big, so we read one chunk at a time
                          low_memory=False, 
                          parse_dates=['CHARTTIME']
             ) # 330712483 rows
labevents = pd.read_csv(f"{MIMIC_DIR}\\LABEVENTS.csv.gz", 
                          usecols=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUENUM'],
                          chunksize=1000000, 
                          low_memory=False, 
                          parse_dates=['CHARTTIME']
             ) #27854055 rows


In [2]:
# https://mimic.mit.edu/docs/iii/tables/patients/
patients.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [3]:
# https://mimic.mit.edu/docs/iii/tables/admissions/
admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [4]:
# https://mimic.mit.edu/docs/iii/tables/diagnoses_icd/
diagnoses_icd.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [5]:
# https://mimic.mit.edu/docs/iii/tables/icustays/
icustays.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [6]:
#https://mimic.mit.edu/docs/iii/tables/microbiologyevents/
micro.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# https://mimic.mit.edu/docs/iii/tables/prescriptions/
prescriptions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [8]:
# https://mimic.mit.edu/docs/iii/tables/chartevents/

# this is just for inspection, to read the first 10 rows
chartevents_preview = pd.read_csv(
    f"{MIMIC_DIR}\\CHARTEVENTS.csv.gz",
    usecols=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUENUM'],
    nrows=10,
    parse_dates=['CHARTTIME']
)
chartevents_preview.head()

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUENUM
0,36,165660,223834,2134-05-12 12:00:00,15.00
1,36,165660,223835,2134-05-12 12:00:00,100.00
2,36,165660,224328,2134-05-12 12:00:00,0.37
3,36,165660,224329,2134-05-12 12:00:00,6.00
4,36,165660,224330,2134-05-12 12:00:00,2.50


In [9]:
# https://mimic.mit.edu/docs/iii/tables/labevents/

# this is just for inspection, to read the first 10 rows
labevents_preview = pd.read_csv(
    f"{MIMIC_DIR}\\LABEVENTS.csv.gz",
    usecols=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUENUM'],
    nrows=10,
    parse_dates=['CHARTTIME']
)
labevents_preview.head()

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUENUM
0,3,NaN,50820,2101-10-12 16:07:00,7.39
1,3,NaN,50800,2101-10-12 18:17:00,NaN
2,3,NaN,50802,2101-10-12 18:17:00,-1.00
3,3,NaN,50804,2101-10-12 18:17:00,22.00
4,3,NaN,50808,2101-10-12 18:17:00,0.93


## Sepsis-3 crteria - Suspected Infection detection

We want to detect the change in SOFA score post suspected infection. Hence we first have to detect the suspected infection onset for the patient. This is calculated by finding out when antibiotics were first administered, or a blood culture was taken within 24 hours of each other (+-) and then taking the earlier of the two times.

In [10]:
# https://mimic.mit.edu/docs/iii/tables/microbiologyevents/
first_culture = micro.groupby('HADM_ID')['CHARTDATE'].min().reset_index()
first_culture.columns = ['HADM_ID', 'first_culture_time']

In [11]:
first_culture

,HADM_ID,first_culture_time
0,100001,2117-09-11
1,100003,2150-04-17
2,100006,2108-04-07
3,100007,2145-03-31
4,100009,2162-05-16
...,...,...
48735,199993,2161-11-02
48736,199994,2188-07-07
48737,199995,2137-12-11
48738,199998,2119-02-22


In [12]:
antibiotics = ['Vancomycin', 'Ceftriaxone', 'Meropenem', 'Piperacillin-Tazobactam', 'Levofloxacin']
extended_abx = [
    'Cefepime', 'Azithromycin', 'Ampicillin', 'Ampicillin-Sulbactam', 
    'Clindamycin', 'Linezolid', 'Metronidazole', 'Cefuroxime', 'Tobramycin', 
    'Gentamicin', 'Imipenem', 'Imipenem-Cilastatin', 'Ciprofloxacin'
]

all_abx = antibiotics + extended_abx

abx_lc = [a.lower() for a in all_abx]
drug_lc = prescriptions['DRUG'].fillna('').str.lower()
# Create a boolean mask: True if any antibiotic name is a substring of the DRUG string
mask = drug_lc.apply(lambda s: any(abx in s for abx in abx_lc))
abx_given = prescriptions[mask]

first_abx = abx_given.groupby('HADM_ID')['STARTDATE'].min().reset_index()
first_abx.columns = ['HADM_ID', 'first_abx_time']

In [13]:
suspected_infection = pd.merge(first_abx, first_culture, on='HADM_ID')
suspected_infection['delta'] = (
    (suspected_infection['first_abx_time'] - suspected_infection['first_culture_time'])
    .dt.total_seconds() / 3600
)

suspected_infection = suspected_infection[suspected_infection['delta'].abs() <= 24]

# Step 5: Set infection time as the earlier of the two
suspected_infection['infection_time'] = suspected_infection[['first_abx_time', 'first_culture_time']].min(axis=1)


In [14]:
suspected_infection.shape

(23746, 5)

## Sepsis 3 criteria - Organ dysfunction markers

In [15]:
# This is for understanding, rather than being relevant to the sofa score calculation
# https://mimic.mit.edu/docs/iii/tables/d_labitems/
labevent_item_ids = {
    "Creatinine": 50912,
    "Platelet Count": 51265,
    "Bilirubin, Total": 50885
}
labevent_items = pd.read_csv(f"{MIMIC_DIR}\\D_LABITEMS.csv.gz", low_memory=False) 

for k,v in labevent_item_ids.items():
    print(labevent_items[labevent_items['ITEMID']==v])


     ROW_ID  ITEMID       LABEL  FLUID   CATEGORY LOINC_CODE
239     113   50912  Creatinine  Blood  Chemistry     2160-0
     ROW_ID  ITEMID           LABEL  FLUID    CATEGORY LOINC_CODE
591     465   51265  Platelet Count  Blood  Hematology      777-3
     ROW_ID  ITEMID             LABEL  FLUID   CATEGORY LOINC_CODE
212      86   50885  Bilirubin, Total  Blood  Chemistry     1975-2


In [16]:
# Labevent based calculations

# ITEMIDs
creat_itemid = 50912
platelet_itemid = 51265
bilirubin_itemid = 50885

# Accumulators
renal_rows = []
coag_rows = []
liver_rows = []

for chunk in labevents:
    chunk = chunk[['HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUENUM']].copy()
    chunk = chunk.dropna(subset=['VALUENUM'])

    chunk['CHARTTIME'] = pd.to_datetime(chunk['CHARTTIME']).dt.floor('h')

    # Renal - Creatinine
    cr = chunk[chunk['ITEMID'] == creat_itemid].copy()
    cr['renal_score'] = pd.cut(
        cr['VALUENUM'],
        bins=[-float('inf'), 1.2, 1.9, 3.4, 4.9, float('inf')], # bins for creatinine
        labels=[0, 1, 2, 3, 4] # assignemnt of severity score
    ).fillna(0).astype(int)
    renal_rows.append(cr[['HADM_ID', 'CHARTTIME', 'renal_score']])

    # Coagulation - Platelets
    plts = chunk[chunk['ITEMID'] == platelet_itemid].copy()
    plts['coag_score'] = pd.cut(
        plts['VALUENUM'],
        bins=[-float('inf'), 20, 49, 99, 149, float('inf')], # bins for platelets
        labels=[4, 3, 2, 1, 0] # assignemnt of severity score
    ).fillna(0).astype(int)
    coag_rows.append(plts[['HADM_ID', 'CHARTTIME', 'coag_score']])

    # Liver - Bilirubin
    bili = chunk[chunk['ITEMID'] == bilirubin_itemid].copy()
    bili['liver_score'] = pd.cut(
        bili['VALUENUM'],
        bins=[-float('inf'), 1.2, 1.9, 5.9, 11.9, float('inf')], # bins for bilirubin
        labels=[0, 1, 2, 3, 4] # assignemnt of severity score
    ).fillna(0).astype(int)
    liver_rows.append(bili[['HADM_ID', 'CHARTTIME', 'liver_score']])

# Concatenate results
renal_scores = pd.concat(renal_rows)
coag_scores = pd.concat(coag_rows)
liver_scores = pd.concat(liver_rows)



In [17]:
renal_scores

,HADM_ID,CHARTTIME,renal_score
15,NaN,2101-10-13 03:00:00,1
28,NaN,2101-10-13 15:00:00,1
42,NaN,2101-10-14 03:00:00,1
86,NaN,2101-10-15 03:00:00,1
106,NaN,2101-10-15 12:00:00,1
...,...,...,...
27853910,113912.0,2105-01-15 06:00:00,0
27853920,107499.0,2115-05-27 04:00:00,0
27853944,107499.0,2115-05-28 04:00:00,0
27853972,120151.0,2115-06-29 04:00:00,1


In [18]:
coag_scores

,HADM_ID,CHARTTIME,coag_score
64,NaN,2101-10-14 03:00:00,0
98,NaN,2101-10-15 03:00:00,0
133,NaN,2101-10-16 04:00:00,0
202,145834.0,2101-10-22 04:00:00,0
365,145834.0,2101-10-23 03:00:00,1
...,...,...,...
27853750,128633.0,2119-12-03 06:00:00,0
27853782,178949.0,2137-08-12 02:00:00,2
27853822,178949.0,2137-08-25 22:00:00,0
27853935,107499.0,2115-05-27 04:00:00,0


In [19]:
# This is for understanding, rather than being relevant to the sofa score calculation
# https://mimic.mit.edu/docs/iii/tables/d_items/

chart_item_ids = {
    "mean arterial pressure (MAP)": [220052], # cardiovascular
    "fraction of inspired oxygen (fio2)": [223835, 3420],
    "glasgow coma scale - eye response": [223900, 220739], 
    "glasgow coma scale - verbal response" : [223901],
    "glasgow coma scale - motor response": [223902],
    
}

d_items = pd.read_csv(f"{MIMIC_DIR}\\D_ITEMS.csv.gz", low_memory=False) 


for k,v in chart_item_ids.items():
    for v_id in v:
        print(k, d_items[d_items['ITEMID']==v_id]["LABEL"])

mean arterial pressure (MAP) 11504    Arterial Blood Pressure mean
Name: LABEL, dtype: object
fraction of inspired oxygen (fio2) 12413    Inspired O2 Fraction
Name: LABEL, dtype: object
fraction of inspired oxygen (fio2) 1838    FIO2
Name: LABEL, dtype: object
glasgow coma scale - eye response 12037    GCS - Verbal Response
Name: LABEL, dtype: object
glasgow coma scale - eye response 11494    GCS - Eye Opening
Name: LABEL, dtype: object
glasgow coma scale - verbal response 12038    GCS - Motor Response
Name: LABEL, dtype: object
glasgow coma scale - motor response 12039    Speech
Name: LABEL, dtype: object


In [20]:
# Central nervous system and respiratory system

fio2_itemids = [223835, 3420]   # FiO2
gcs_itemids = {
    'EYE': [223900, 220739],    # Eye Response
    'VERBAL': [223901],         # Verbal Response
    'MOTOR': [223902],          # Motor Response
}

fio2_rows = []
gcs_rows = []


for chunk in chartevents:
    chunk = chunk[['HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUENUM']].copy()
    chunk = chunk.dropna(subset=['VALUENUM'])
    chunk['CHARTTIME'] = pd.to_datetime(chunk['CHARTTIME']).dt.floor('h')

    ## --- Respiratory: FiO2 ---
    fio2_chunk = chunk[chunk['ITEMID'].isin(fio2_itemids)].copy()
    fio2_chunk.rename(columns={'VALUENUM': 'fio2'}, inplace=True)
    fio2_rows.append(fio2_chunk[['HADM_ID', 'CHARTTIME', 'fio2']])

    ## --- CNS: GCS Scores ---
    gcs_chunk = chunk[chunk['ITEMID'].isin(
        gcs_itemids['EYE'] + gcs_itemids['VERBAL'] + gcs_itemids['MOTOR']
    )].copy()
    gcs_rows.append(gcs_chunk)

gcs_all = pd.concat(gcs_rows)
gcs_all = gcs_all.dropna(subset=['VALUENUM'])
gcs_all['CHARTTIME'] = pd.to_datetime(gcs_all['CHARTTIME']).dt.floor('h')

# Pivot: get one column per GCS component
gcs_pivot = gcs_all.pivot_table(index=['HADM_ID', 'CHARTTIME'], 
                                columns='ITEMID', values='VALUENUM', aggfunc='max').reset_index()
gcs_pivot.columns.name = None

# Rename columns for clarity
gcs_pivot = gcs_pivot.rename(columns={
    223900: 'eye',
    220739: 'eye_alt',
    223901: 'verbal',
    223902: 'motor'
})

# Fill missing eye from alternate
gcs_pivot['eye'] = gcs_pivot['eye'].combine_first(gcs_pivot['eye_alt'])

# Ensure all three components exist
for col in ['eye', 'verbal', 'motor']:
    if col not in gcs_pivot.columns:
        gcs_pivot[col] = np.nan

# Total GCS calculation (NaNs allowed)
gcs_pivot['gcs_total'] = gcs_pivot[['eye', 'verbal', 'motor']].sum(axis=1, min_count=1)

# Fill fully missing GCS with 15 (assume alert) 
# TODO alternative - forward fill
gcs_pivot['gcs_total'] = gcs_pivot['gcs_total'].fillna(15)

# CNS SOFA scoring
gcs_pivot['cns_score'] = pd.cut(
    gcs_pivot['gcs_total'],
    bins=[-float('inf'), 5, 8, 11, 14, 15],
    labels=[4, 3, 2, 1, 0]
).astype(int)

# Final GCS score output
cns_scores = gcs_pivot[['HADM_ID', 'CHARTTIME', 'cns_score']]

# Respiratory system P/F ratio

fio2_df = pd.concat(fio2_rows)

# labevents reset
labevents = pd.read_csv(f"{MIMIC_DIR}\\LABEVENTS.csv.gz", 
                          usecols=['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUENUM'],
                          chunksize=1000000, 
                          low_memory=False, 
                          parse_dates=['CHARTTIME']
             ) #27854055 rows

pao2_itemid = 50821
resp_chunks = []

for chunk in labevents:
    pao2 = chunk[chunk['ITEMID'] == pao2_itemid][['HADM_ID', 'CHARTTIME', 'VALUENUM']].copy()
    pao2 = pao2.dropna(subset=['VALUENUM'])

    pao2['CHARTTIME'] = pd.to_datetime(pao2['CHARTTIME']).dt.floor('h')
    pao2.rename(columns={'VALUENUM': 'pao2'}, inplace=True)
    resp_chunks.append(pao2)

pao2_df = pd.concat(resp_chunks)


# Merge PaO2 and FiO2
resp_merged = pd.merge(pao2_df, fio2_df, how='inner', on=['HADM_ID', 'CHARTTIME'])

# Avoid divide-by-zero
resp_merged = resp_merged[resp_merged['fio2'] > 0]

# Compute P/F ratio
resp_merged['p_f_ratio'] = resp_merged['pao2'] / resp_merged['fio2']

# Assign respiratory SOFA score
resp_merged['resp_score'] = pd.cut(
    resp_merged['p_f_ratio'],
    bins=[-float('inf'), 100, 200, 300, 400, float('inf')], # bins for P/F ratio
    labels=[4, 3, 2, 1, 0] # assignemnt of severity score
).fillna(0).astype(int)

resp_scores = resp_merged[['HADM_ID', 'CHARTTIME', 'resp_score']]


In [21]:
resp_scores

,HADM_ID,CHARTTIME,resp_score
0,106266.0,2114-11-29 22:00:00,4
1,106266.0,2114-11-30 00:00:00,4
2,106266.0,2114-11-30 14:00:00,4
3,106266.0,2114-11-30 16:00:00,4
4,106266.0,2114-11-30 16:00:00,4
...,...,...,...
81230,194096.0,2118-10-19 11:00:00,4
81231,178949.0,2137-08-12 08:00:00,4
81232,178949.0,2137-08-26 04:00:00,4
81233,103219.0,2109-12-29 20:00:00,4


In [23]:
# cardio vascular system 
from cardiovascular_score import cardio_score
weights = pd.read_csv("weights_hadm_id.csv")
cardio_scores = cardio_score(weights)

cardio_scores = cardio_scores[['HADM_ID', 'CHARTTIME', 'cardio_score']]

C:\Users\amits\work\github.com\amitsaha\ml-fairness-health\mywork\localized-updating\experiments\6h-prior-onset\feature-prep\cardiovascular_score.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['drug'] = name
C:\Users\amits\work\github.com\amitsaha\ml-fairness-health\mywork\localized-updating\experiments\6h-prior-onset\feature-prep\cardiovascular_score.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['drug'] = name
C:\Users\amits\work\github.com\amitsaha\ml-fairness-health\mywork\localized-up

In [24]:
cardio_scores

,HADM_ID,CHARTTIME,cardio_score
0,100003.0,2150-04-17 17:00:00,1
1,100038.0,NaT,1
2,100053.0,2124-07-14 04:00:00,1
3,100061.0,2178-12-25 14:30:00,1
4,100068.0,2192-01-13 16:39:00,1
...,...,...,...
10499,199969.0,2201-01-27 19:55:00,1
10500,199972.0,2186-08-29 13:18:00,1
10501,199992.0,2155-02-11 17:21:00,1
10502,199993.0,2161-10-23 22:00:00,1


In [25]:
from functools import reduce
score_dfs = [
    renal_scores,    
    coag_scores,
    liver_scores,
    cardio_scores,
    cns_scores,
    resp_scores
]

# Merge all on HADM_ID + CHARTTIME
sofa_df = reduce(lambda left, right: pd.merge(left, right, how='outer', on=['HADM_ID', 'CHARTTIME']), score_dfs)

# Fill missing scores with 0
for col in sofa_df.columns:
    if '_score' in col:
        sofa_df[col] = sofa_df[col].fillna(0).astype(int)

# Calculate total SOFA score
sofa_df['total_sofa_score'] = sofa_df[[col for col in sofa_df.columns if '_score' in col]].sum(axis=1)

In [26]:
sofa_df

,HADM_ID,CHARTTIME,renal_score,coag_score,liver_score,cardio_score,cns_score,resp_score,total_sofa_score
0,100001.0,2117-09-11 08:00:00,2,0,0,0,0,0,2
1,100001.0,2117-09-11 10:00:00,2,0,0,0,0,0,2
2,100001.0,2117-09-11 13:00:00,0,0,0,0,2,0,2
3,100001.0,2117-09-11 14:00:00,2,0,0,0,0,0,2
4,100001.0,2117-09-11 16:00:00,0,0,0,0,2,0,2
...,...,...,...,...,...,...,...,...,...
1599872,NaN,2207-11-20 11:00:00,1,0,0,0,0,0,1
1599873,NaN,2208-01-03 15:00:00,0,0,0,0,0,0,0
1599874,NaN,2208-01-04 05:00:00,0,0,0,0,0,0,0
1599875,NaN,2208-01-05 05:00:00,0,0,0,0,0,0,0


In [27]:
# sepsis onset

# Ensure timestamps are datetime
sofa_df['CHARTTIME'] = pd.to_datetime(sofa_df['CHARTTIME'])

# Merge SOFA with infection time
merged = pd.merge(sofa_df, suspected_infection[['HADM_ID', 'infection_time']], on='HADM_ID', how='inner')

# Time diff in hours
merged['time_diff_hours'] = (merged['CHARTTIME'] - merged['infection_time']).dt.total_seconds() / 3600

# Pre- and post-infection windows
pre_window = merged[(merged['time_diff_hours'] >= -24) & (merged['time_diff_hours'] < 0)]
post_window = merged[(merged['time_diff_hours'] >= 0) & (merged['time_diff_hours'] <= 48)]

# Get baseline SOFA (lowest in 24h before)
baseline_sofa = pre_window.groupby('HADM_ID')['total_sofa_score'].min().reset_index()
baseline_sofa.columns = ['HADM_ID', 'baseline_sofa']

# Join baseline with post-infection SOFA
post_with_baseline = pd.merge(post_window, baseline_sofa, on='HADM_ID', how='left')
post_with_baseline['sofa_delta'] = post_with_baseline['total_sofa_score'] - post_with_baseline['baseline_sofa']

# First time SOFA delta ≥ 2 = Sepsis Onset
sepsis_onset = (
    post_with_baseline[post_with_baseline['sofa_delta'] >= 2]
    .sort_values(['HADM_ID', 'CHARTTIME'])
    .groupby('HADM_ID')
    .first()
    .reset_index()
)

# Output: earliest time of sepsis
sepsis_onset = sepsis_onset[['HADM_ID', 'CHARTTIME']].rename(columns={'CHARTTIME': 'sepsis_onset_time'})


In [31]:
sepsis_onset.shape

(1731, 2)

In [33]:
sepsis_onset.to_csv("sepsis_onsets_updated.csv")